In [2]:
import math
import numpy as np

In [3]:
def para_tableau(c, A, b):
    """
    Função para montar o tableau inicial combinando a matriz A, vetor b e coeficientes c.
    """
    tableau = [row + [bi] for row, bi in zip(A, b)]
    # Linha da função objetivo no tableau
    tableau.append(c + [0])
    return tableau



def checar_otima(tableau):
    z = tableau[-1]
    found = False
    for x in z[:-1]:
        if x > 0:
            found = True
            break
    return found




"""
def checar_otima(tableau):
    #Verifica se a solução é ótima. Se algum coeficiente na linha de custo (função objetivo) é negativo, ainda não é ótima.
    
    # original: return any(x > 0 for x in tableau[-1][:-1])
    return not any(x < 0 for x in tableau[-1][:-1])
"""



def obter_posicao_pivo(tableau):
    """
    Encontra a posição do pivô. Retorna a linha e coluna que determinam a entrada e saída das variáveis.
    """
    # Escolher a coluna da variável que entra na base (primeiro elemento positivo)
    coluna_pivo = next(i for i, x in enumerate(tableau[-1][:-1]) if x > 0)
    
    # Escolher a linha do pivô usando o critério de razão mínima
    restricoes = [(tableau[i][-1] / tableau[i][coluna_pivo] if tableau[i][coluna_pivo] > 0 else math.inf) for i in range(len(tableau) - 1)]
    linha_pivo = restricoes.index(min(restricoes))
    return linha_pivo, coluna_pivo

def passo_pivo(tableau, posicao_pivo):
    """
    Executa o passo de pivoteamento no tableau, transformando-o conforme a nova base.
    """
    i, j = posicao_pivo
    valor_pivo = tableau[i][j]
    
    # Divide a linha pivô pelo valor do pivô
    tableau[i] = [x / valor_pivo for x in tableau[i]]
    
    # Atualizar as outras linhas
    for k, linha in enumerate(tableau):
        if k != i:
            multiplicador = linha[j]
            tableau[k] = [x - multiplicador * pivot_element for x, pivot_element in zip(linha, tableau[i])]
    return tableau

def eh_basica(coluna):
    """
    Verifica se uma coluna é uma coluna de variável básica (contém apenas um valor 1 e o restante 0).
    """
    # Conta o número de zeros e de uns na coluna
    return np.count_nonzero(coluna == 0) == len(coluna) - 1 and np.count_nonzero(coluna == 1) == 1

In [4]:
def obter_solucao(tableau):
    """
    Extrai a solução das variáveis básicas a partir do tableau final.
    """
    colunas = np.array(tableau).T
    solucao = []
    for coluna in colunas[:-1]:  # Última coluna é o valor da função objetivo
        if eh_basica(coluna):
            linha = coluna.tolist().index(1)
            solucao.append(tableau[linha][-1])
        else:
            solucao.append(0)
    return solucao

def simplex(c, A, b):
    """
    Método Simplex para resolver o problema de programação linear.
    """
    tableau = para_tableau(c, A, b)

    # Executa o pivoteamento enquanto não encontra a solução ótima
    while checar_otima(tableau):
        posicao_pivo = obter_posicao_pivo(tableau)
        tableau = passo_pivo(tableau, posicao_pivo)

    return obter_solucao(tableau)

In [5]:
def ler_variaveis():
    """
    Lê interativamente os valores de c, A e b do problema de programação linear.
    """
    try:
        # Leitura da função objetivo
        c = list(map(float, input("Digite os coeficientes da função objetivo (separados por espaço): ").split()))
        
        # Leitura da matriz de coeficientes das restrições
        num_restricoes = int(input("Digite o número de restrições: "))
        A = []
        for i in range(num_restricoes):
            linha = list(map(float, input(f"Digite os coeficientes da restrição {i+1} (separados por espaço): ").split()))
            if len(linha) != len(c):
                raise ValueError("O número de coeficientes na restrição não corresponde ao número de variáveis.")
            A.append(linha)
        
        # Leitura dos termos independentes das restrições
        b = list(map(float, input("Digite os termos independentes das restrições (separados por espaço): ").split()))
        if len(b) != num_restricoes:
            raise ValueError("O número de termos independentes não corresponde ao número de restrições.")
        
        return c, A, b
    
    except ValueError as ve:
        print(f"Erro de valor: {ve}")
    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")


In [9]:
# Le as variaveis relacionadas a função objetivo:
"""
c = coeficientes da função objetivo
A = matriz de coeficientes das restrições
b = vetor de termos independentes
"""
#c, A, b = ler_variaveis()

# coeficientes da função objetivo
c = [1, 1, 0, 0, 0]

# matriz de coeficientes das restrições
A = [
    [-1, 1, 1, 0, 0],
    [ 1, 0, 0, 1, 0],
    [ 0, 1, 0, 0, 1]
]

# vetor de termos independentes das restrições
b = [2, 4, 4]



# Teste do algoritmo com o exemplo fornecido
solucao = simplex(c, A, b)
print("Solução ótima encontrada:", solucao)


Solução ótima encontrada: [4.0, 4.0, 2.0, 0, 0]
